# COMP34812 Natural Language Understanding Courseworklow key lemming an stemming


## Install required packages

In [1]:
!pip install tensorflow pandas nltk numpy matplotlib scikit-learn sentencepiece tokenizers
!python -m spacy download en_core_web_sm


In [35]:
import pandas as pd
import regex as re
import numpy as np
import nltk
import os
import tensorflow as tf
import spacy

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

from nltk.corpus import stopwords
from tokenizers import ByteLevelBPETokenizer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [36]:
print("GPUs:", tf.config.list_physical_devices('GPU'))

nlp = spacy.load("en_core_web_sm")

GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [17]:
from google.colab import drive
drive.mount('/content/drive')
if not os.path.exists('glove_embeddings'):
  !mkdir glove_embeddings
  !cp /content/drive/MyDrive/glove.6B.200d.txt glove_embeddings/


Mounted at /content/drive
cp: cannot create regular file 'glove_embeddings/': Not a directory


In [15]:
if not os.path.exists('glove_embeddings'):
  !wget http://nlp.stanford.edu/data/glove.6B.zip -O glove.zip
  !unzip -j glove.zip "glove.6B.200d.txt" -d glove_embeddings

--2025-03-25 22:33:25--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-03-25 22:33:26--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-03-25 22:33:26--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.zip’

glove.z

## Load dataset

In [4]:
dev_set = pd.read_csv('dev.csv')
dev_set.head()

,premise,hypothesis,label
0,"By starting at the soft underbelly, the 16,000...","General Nelson A. Miles had 30,000 troops in h...",0
1,"The class had broken into a light sweat, but w...",The class grew more tense as time went on.,1
2,"Samson had his famous haircut here, but he wou...",It was unknown where exactly within the town S...,1
3,A man with a black shirt holds a baby while a ...,A darkly dressed man passes a crying baby to a...,0
4,I know that many of you are interested in addr...,The problems must be addressed,1


In [18]:
train_set = pd.read_csv('train.csv')
train_set.head()

,premise,hypothesis,label
0,yeah i don't know cut California in half or so...,Yeah. I'm not sure how to make that fit. Maybe...,1
1,actual names will not be used,"For the sake of privacy, actual names are not ...",1
2,The film was directed by Randall Wallace.,The film was directed by Randall Wallace and s...,1
3,"""How d'you know he'll sign me on?""Anse studie...",Anse looked at himself in a cracked mirror.,1
4,In the light of the candles his cheeks looked ...,Drew regarded his best friend and noted that i...,1


In [19]:
stop_words = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

def clean_text(text):
    text = str(text)

    text = text.lower()

    text = re.sub(r'[^\w\s]', ' ', text)

    text = nltk.word_tokenize(text)

    processed = []
    for word in text:
        if word in stop_words:
            continue

        word = lemmatizer.lemmatize(word)

        word = word.strip()

        if len(word) < 2:
            continue

        processed.append(word)

    return processed

In [20]:
dev_set['premise_tokens'] = dev_set['premise'].apply(clean_text)
dev_set['hypothesis_tokens'] = dev_set['hypothesis'].apply(clean_text)

train_set['premise_tokens'] = train_set['premise'].apply(clean_text)
train_set['hypothesis_tokens'] = train_set['hypothesis'].apply(clean_text)

In [21]:
dev_set.head()

,premise,hypothesis,label,premise_tokens,hypothesis_tokens,combined_text
0,"By starting at the soft underbelly, the 16,000...","General Nelson A. Miles had 30,000 troops in h...",0,"[starting, soft, underbelly, 16, 000, troop, g...","[general, nelson, mile, 30, 000, troop, attack]",starting soft underbelly 16 000 troop general ...
1,"The class had broken into a light sweat, but w...",The class grew more tense as time went on.,1,"[class, broken, light, sweat, gasping, air]","[class, grew, tense, time, went]",class broken light sweat gasping air class gre...
2,"Samson had his famous haircut here, but he wou...",It was unknown where exactly within the town S...,1,"[samson, famous, haircut, would, find, hard, r...","[unknown, exactly, within, town, samson, recei...",samson famous haircut would find hard recogniz...
3,A man with a black shirt holds a baby while a ...,A darkly dressed man passes a crying baby to a...,0,"[man, black, shirt, hold, baby, blue, shirted,...","[darkly, dressed, man, pass, cry, baby, man, l...",man black shirt hold baby blue shirted individ...
4,I know that many of you are interested in addr...,The problems must be addressed,1,"[know, many, interested, addressing, issue, le...","[problem, must, addressed]",know many interested addressing issue legislat...


In [22]:
train_set.head()

,premise,hypothesis,label,premise_tokens,hypothesis_tokens
0,yeah i don't know cut California in half or so...,Yeah. I'm not sure how to make that fit. Maybe...,1,"[yeah, know, cut, california, half, something]","[yeah, sure, make, fit, maybe, could, cut, cal..."
1,actual names will not be used,"For the sake of privacy, actual names are not ...",1,"[actual, name, used]","[sake, privacy, actual, name, used]"
2,The film was directed by Randall Wallace.,The film was directed by Randall Wallace and s...,1,"[film, directed, randall, wallace]","[film, directed, randall, wallace, star, mel, ..."
3,"""How d'you know he'll sign me on?""Anse studie...",Anse looked at himself in a cracked mirror.,1,"[know, sign, anse, studied, unkempt, clean, re...","[anse, looked, cracked, mirror]"
4,In the light of the candles his cheeks looked ...,Drew regarded his best friend and noted that i...,1,"[light, candle, cheek, looked, even, hollow, t...","[drew, regarded, best, friend, noted, light, l..."


Dataset analysis

In [23]:
# Randomly select 90% of the rows for the training set
train_df = train_set.sample(frac=0.9, random_state=42)
# The rest of the rows will form the test set
test_df = train_set.drop(train_df.index)
train_df.head()



,premise,hypothesis,label,premise_tokens,hypothesis_tokens
2166,"No, he waits until he has had a violent quarre...",He was clear and free from suspicion.,0,"[wait, violent, quarrel, whole, household, cog...","[clear, free, suspicion]"
10333,oh yeah but that's that's a good way to make a...,A marriage can work when you do dishonest thin...,0,"[oh, yeah, good, way, make, big, problem, marr...","[marriage, work, dishonest, thing]"
10835,Land that is acquired for or in connection wit...,Land that is acquired has no connection with i...,0,"[land, acquired, connection, item, general, pp...","[land, acquired, connection, item, general, pp]"
4688,and my wife is from Plains if you know where P...,My wife went to school in Plains.,1,"[wife, plain, know, plain]","[wife, went, school, plain]"
17561,Since 1980 the building has housed the Museum ...,The building has never been able to house any ...,0,"[since, 1980, building, housed, museum, ethnog...","[building, never, able, house, museum]"


# NER (might use)


In [24]:

def ner_features(text1, text2):
    doc1, doc2 = nlp(text1), nlp(text2)
    ents1 = set(ent.text.lower() for ent in doc1.ents)
    ents2 = set(ent.text.lower() for ent in doc2.ents)
    overlap = len(ents1 & ents2) / (len(ents1 | ents2) + 1e-5)
    return np.array([overlap])


# TF-IDF word embeddings

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert tokenized text back to strings for TF-IDF processing
train_premise_texts = train_set['premise_tokens'].apply(lambda x: ' '.join(x))
train_hypothesis_texts = train_set['hypothesis_tokens'].apply(lambda x: ' '.join(x))

dev_premise_texts = dev_set['premise_tokens'].apply(lambda x: ' '.join(x))
dev_hypothesis_texts = dev_set['hypothesis_tokens'].apply(lambda x: ' '.join(x))

# Combine premises & hypotheses for TF-IDF training
all_texts = pd.concat([train_premise_texts, train_hypothesis_texts])

# Initialize and fit TF-IDF on training data
tfidf = TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
tfidf.fit(all_texts)

# Transform training and dev sets
train_premise_tfidf = tfidf.transform(train_premise_texts).toarray()
train_hypothesis_tfidf = tfidf.transform(train_hypothesis_texts).toarray()

dev_premise_tfidf = tfidf.transform(dev_premise_texts).toarray()
dev_hypothesis_tfidf = tfidf.transform(dev_hypothesis_texts).toarray()



# Glove embeddings

In [27]:
glove = "./glove_embeddings/glove.6B.200d.txt"
def load_glove(glove_file):
    embeddings_dict = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype=np.float32)  # <-- Convert to float32
            embeddings_dict[word] = vector
    return embeddings_dict

embedding_dim = 200
loaded_glove = load_glove(glove)



In [28]:
from sklearn.metrics.pairwise import cosine_similarity
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-5)


In [29]:
#Traidntiona emmedings

def sentence_embedding(tokens, embeddings_dict, embedding_dim):
    valid_embeddings = [embeddings_dict[token] for token in tokens if token in embeddings_dict]
    if not valid_embeddings:
        return np.zeros(embedding_dim)
    return np.mean(valid_embeddings, axis=0)

def pairwise_embedding(premise_tokens, hypothesis_tokens,  embeddings_dict,embedding_dim):
    prem_emb = sentence_embedding(premise_tokens, embeddings_dict, embedding_dim)
    hyp_emb = sentence_embedding(hypothesis_tokens, embeddings_dict, embedding_dim)
    ner_feat = ner_features(train_set['premise'], train_set['hypothesis'])
    cos_sim = np.array([cosine_similarity(prem_emb, hyp_emb)])

    combined = np.concatenate([
        prem_emb,
        hyp_emb,
        np.abs(prem_emb - hyp_emb),
        ner_feat,
        cos_sim
    ]).astype(np.float32)

    return combined



In [30]:
"""
import numpy as np
from tqdm import tqdm
tqdm.pandas()

# Convert list of numpy arrays into a single 2D numpy array
train_embeddings = np.vstack(train_set.apply(lambda row: create_embedding(row, loaded_glove, embedding_dim), axis=1))
dev_embeddings = np.vstack(dev_set.apply(lambda row: create_embedding(row, loaded_glove, embedding_dim), axis=1))
train_labels = train_set['label'].values
dev_labels = dev_set['label'].values
"""



In [ ]:
"""
train_embeddings_tf = tf.convert_to_tensor(train_embeddings, dtype=tf.float32)
dev_embeddings_tf = tf.convert_to_tensor(dev_embeddings, dtype=tf.float32)

train_labels_tf = tf.convert_to_tensor(train_labels, dtype=tf.int32)
dev_labels_tf = tf.convert_to_tensor(dev_labels, dtype=tf.int32)
"""

# tokinizer LSTM tokinizer

In [ ]:
def tokens_to_embedding_sequence(tokens, glove, dim):
    return [glove.get(tok, np.zeros(dim)) for tok in tokens]

In [ ]:
premise_seq = tokens_to_embedding_sequence(row['premise_tokens'], glove, dim)
hypothesis_seq = tokens_to_embedding_sequence(row['hypothesis_tokens'], glove, dim)
combined_seq = premise_seq + hypothesis_seq


#LSTM RNN appaorch


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional

model = Sequential([
    Bidirectional(LSTM(128, return_sequences=False), input_shape=(max_len, embedding_dim)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
model.fit(train_embeddings_tf, train_labels_tf,
          validation_data=(dev_embeddings_tf, dev_labels_tf),
          epochs=10, batch_size=64)


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

preds = model.predict(dev_embeddings_tf)
pred_labels = np.argmax(preds, axis=1)

print(classification_report(dev_labels_tf, pred_labels, target_names=['entailment', 'contradiction']))


# Traditional Approach

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

"""
param_grid = {
    'C': [0.01, 0.1, 0.2],  # Regularization strength
    'solver': ['lbfgs','saga','newton-cg','sag' ],  # Different solvers for logistic regression
    'max_iter': [500, 1000]  # More iterations for convergence
}
"""

scaler = StandardScaler()
train_emb_scaled = scaler.fit_transform(train_embeddings)
dev_emb_scaled = scaler.transform(dev_embeddings)

param_grid = {
    'C': [0.2,0.1],  # Regularization strength
    'solver': ['saga','newton-cg','sag' ],  # Different solvers for logistic regression
    'max_iter': [200, 300, ]  # More iterations for convergence
}

clf = GridSearchCV(LogisticRegression(), param_grid, cv=3, scoring='f1_macro', verbose=1, n_jobs=-1)
#clf = (solver='lbfgs', 'C'= 0.01, max_iter=500, cv=3, scoring='f1_macro', verbose=1, n_jobs=-1)
clf.fit(train_emb_scaled, train_set["label"].values)  # Train on enhanced embeddings

# Print Best Parameters
print("Best Parameters:", clf.best_params_)

# Evaluate on validation set
preds = clf.best_estimator_.predict(dev_emb_scaled)
print(classification_report(dev_set['label'].values, preds, target_names=['entailment', 'contradiction']))


Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best Parameters: {'C': 0.2, 'max_iter': 300, 'solver': 'sag'}
               precision    recall  f1-score   support

   entailment       0.58      0.52      0.55      3258
contradiction       0.59      0.65      0.62      3478

     accuracy                           0.59      6736
    macro avg       0.59      0.59      0.59      6736
 weighted avg       0.59      0.59      0.59      6736



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
